# Borisov Dmitriy 6133 Lab2

## Инициализация спарка

In [1]:
import findspark
findspark.init()

In [2]:
from datetime import datetime
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, DateType

In [3]:
 from pyspark.sql import SparkSession
import os

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.13.0 pyspark-shell'

spark_session = SparkSession\
    .builder\
    .getOrCreate()

In [4]:
spark_session

In [5]:
sc = spark_session._sc

## Проверка существования файлов

In [6]:
!head /home/workspace/data/programming-languages.csv

name,wikipedia_url
A# .NET,https://en.wikipedia.org/wiki/A_Sharp_(.NET)
A# (Axiom),https://en.wikipedia.org/wiki/A_Sharp_(Axiom)
A-0 System,https://en.wikipedia.org/wiki/A-0_System
A+,https://en.wikipedia.org/wiki/A%2B_(programming_language)
A++,https://en.wikipedia.org/wiki/A%2B%2B
ABAP,https://en.wikipedia.org/wiki/ABAP
ABC,https://en.wikipedia.org/wiki/ABC_(programming_language)
ABC ALGOL,https://en.wikipedia.org/wiki/ABC_ALGOL
ABSET,https://en.wikipedia.org/wiki/ABSET


In [7]:
!head -n 3 /home/workspace/data/posts_sample.xml

���<?xml version="1.0" encoding="utf-8"?>
<posts>
  <row Id="4" PostTypeId="1" AcceptedAnswerId="7" CreationDate="2008-07-31T21:42:52.667" Score="630" ViewCount="42817" Body="&lt;p&gt;I want to use a track-bar to change a form's opacity.&lt;/p&gt;&#xA;&#xA;&lt;p&gt;This is my code:&lt;/p&gt;&#xA;&#xA;&lt;pre&gt;&lt;code&gt;decimal trans = trackBar1.Value / 5000;&#xA;this.Opacity = trans;&#xA;&lt;/code&gt;&lt;/pre&gt;&#xA;&#xA;&lt;p&gt;When I build the application, it gives the following error:&lt;/p&gt;&#xA;&#xA;&lt;blockquote&gt;&#xA;  &lt;p&gt;Cannot implicitly convert type &lt;code&gt;'decimal'&lt;/code&gt; to &lt;code&gt;'double'&lt;/code&gt;&lt;/p&gt;&#xA;&lt;/blockquote&gt;&#xA;&#xA;&lt;p&gt;I tried using &lt;code&gt;trans&lt;/code&gt; and &lt;code&gt;double&lt;/code&gt; but then the control doesn't work. This code worked fine in a past VB.NET project.&lt;/p&gt;&#xA;" OwnerUserId="8" LastEditorUserId="3641067" LastEditorDisplayName="Rich B" LastEditDate="2019-07-19T01:39:54.173" 

## Чтение файлов

In [8]:
 prog_lang_schema = StructType([
    StructField("name", StringType(), False),
    StructField("wikipedia_url", StringType(), False),
])

In [9]:
langs = spark_session.read.csv("file:///home/workspace/data/programming-languages.csv", schema=prog_lang_schema)

In [10]:
langs.head(5)

[Row(name='name', wikipedia_url='wikipedia_url'),
 Row(name='A# .NET', wikipedia_url='https://en.wikipedia.org/wiki/A_Sharp_(.NET)'),
 Row(name='A# (Axiom)', wikipedia_url='https://en.wikipedia.org/wiki/A_Sharp_(Axiom)'),
 Row(name='A-0 System', wikipedia_url='https://en.wikipedia.org/wiki/A-0_System'),
 Row(name='A+', wikipedia_url='https://en.wikipedia.org/wiki/A%2B_(programming_language)')]

In [11]:
langs_list = [str(row[0]) for row in langs.select('name').collect()]

In [12]:
langs_list[:5]

['name', 'A# .NET', 'A# (Axiom)', 'A-0 System', 'A+']

In [13]:
!ls data

README.md			   nyctaxi.csv		      trips.csv
list_of_countries_sorted_gini.txt  posts_sample.xml	      warandsociety.txt
nycTaxiFares.gz			   programming-languages.csv
nycTaxiRides.gz			   stations.csv


In [14]:
posts_sample = spark_session.read.format("xml").options(rowTag="row").load('file:///home/workspace/data/posts_sample.xml')

In [15]:
posts_sample.head(2)

[Row(_AcceptedAnswerId=7, _AnswerCount=13, _Body="<p>I want to use a track-bar to change a form's opacity.</p>\n\n<p>This is my code:</p>\n\n<pre><code>decimal trans = trackBar1.Value / 5000;\nthis.Opacity = trans;\n</code></pre>\n\n<p>When I build the application, it gives the following error:</p>\n\n<blockquote>\n  <p>Cannot implicitly convert type <code>'decimal'</code> to <code>'double'</code></p>\n</blockquote>\n\n<p>I tried using <code>trans</code> and <code>double</code> but then the control doesn't work. This code worked fine in a past VB.NET project.</p>\n", _ClosedDate=None, _CommentCount=2, _CommunityOwnedDate=datetime.datetime(2012, 10, 31, 16, 42, 47, 213000), _CreationDate=datetime.datetime(2008, 7, 31, 21, 42, 52, 667000), _FavoriteCount=48, _Id=4, _LastActivityDate=datetime.datetime(2019, 7, 19, 1, 39, 54, 173000), _LastEditDate=datetime.datetime(2019, 7, 19, 1, 39, 54, 173000), _LastEditorDisplayName='Rich B', _LastEditorUserId=3641067, _OwnerDisplayName=None, _OwnerUs

## Обработка данных

In [16]:
def define_language(row):
    language_tag = None
    for lang in langs_list:
        if '<' + lang.lower() + '>' in row._Tags.lower():
            language_tag = lang
            break
    if language_tag is None:
        return None
    return (row._Id, language_tag)

In [17]:
def check_date_range(row, year):
    left_border = datetime(year=year, month=1, day=1)
    right_border = datetime(year=year, month=12, day=31)
    created_date = row._CreationDate
    return created_date >= left_border and created_date <= right_border

In [18]:
top_languages_array = {}
years = range(2010, 2020)
for year in years:
    top_languages_array[year] = posts_sample.rdd\
        .filter(lambda row: row._Tags is not None and check_date_range(row, year))\
        .map(define_language)\
        .filter(lambda row: row is not None)\
        .keyBy(lambda row: row[1])\
        .aggregateByKey(
            0,
            lambda acc, value: acc + 1,
            lambda acc1, acc2: acc1 + acc2,
        )\
        .sortBy(lambda row: row[1], ascending=False)\
        .toDF()
    top_languages_array[year] = top_languages_array[year].select(col('_1').alias(f'Programming_Languages'), 
                                                                 col('_2').alias(f'Posts_of_{year}'))
    top_languages_array[year].show()

+---------------------+-------------+
|Programming_Languages|Posts_of_2010|
+---------------------+-------------+
|                 Java|           52|
|           JavaScript|           44|
|                  PHP|           42|
|               Python|           25|
|          Objective-C|           22|
|                    C|           20|
|                 Ruby|           11|
|               Delphi|            7|
|                    R|            3|
|                 Bash|            3|
|          AppleScript|            3|
|                 Perl|            3|
|                   F#|            2|
|              Haskell|            2|
|                BASIC|            1|
|                   Go|            1|
|                Mouse|            1|
|                XPath|            1|
|         ActionScript|            1|
|               MATLAB|            1|
+---------------------+-------------+
only showing top 20 rows

+---------------------+-------------+
|Programming_Languages|P

## Запись результатов

In [19]:
parquetFiles = []
for year in top_languages_array.keys():
    top_languages_array[year].write.parquet(f"/My_parquets/top_programming_languages_of_{year}.parquet")
    parquetFile = spark_session.read.parquet(f"/My_parquets/top_programming_languages_of_{year}.parquet")
    parquetFile.createOrReplaceTempView(f"parquetFile_{year}")
    parquetFiles.append(parquetFile)

In [20]:
for i in range(len(years)):
    print(f"Top 10 of {years[i]}")
    spark_session.sql(f"SELECT Programming_Languages FROM parquetFile_{years[i]} ORDER BY Posts_of_{years[i]} DESC LIMIT 10")\
        .show()

Top 10 of 2010
+---------------------+
|Programming_Languages|
+---------------------+
|                 Java|
|           JavaScript|
|                  PHP|
|               Python|
|          Objective-C|
|                    C|
|                 Ruby|
|               Delphi|
|                 Bash|
|          AppleScript|
+---------------------+

Top 10 of 2011
+---------------------+
|Programming_Languages|
+---------------------+
|                  PHP|
|                 Java|
|           JavaScript|
|               Python|
|          Objective-C|
|                    C|
|                 Ruby|
|               Delphi|
|                 Perl|
|                 Bash|
+---------------------+

Top 10 of 2012
+---------------------+
|Programming_Languages|
+---------------------+
|                  PHP|
|           JavaScript|
|                 Java|
|               Python|
|          Objective-C|
|                    C|
|                 Ruby|
|                 Bash|
|                